In [32]:
import pandas as pd 
import numpy as np

In [2]:
from simpledbf import Dbf5


In [84]:
catminde = Dbf5("defunciones_base_datos_2021_dbf/CATMINDE.dbf",codec="latin-1")
catminde = catminde.to_dataframe()
dic_catminde = catminde.set_index("CVE").DESCRIP.to_dict()

In [86]:
catminde

,CVE,DESCRIP
0,A010,Fiebre tifoidea
1,A014,"Fiebre paratifoidea, no especificada"
2,A020,Enteritis debida a Salmonella
3,A021,Sepsis debida a Salmonella
4,A029,"Infección debida a Salmonella, no especificada"
...,...,...
3984,Y859,"Secuelas de otros accidentes de transporte, y ..."
3985,Y86X,Secuelas de otros accidentes
3986,Y870,Secuelas de lesiones autoinfligidas
3987,Y880,Secuelas de efectos adversos causados por drog...


In [3]:
dbf = Dbf5("defunciones_base_datos_2021_dbf/defun21.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO","CAUSA_DEF","DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]


In [25]:
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



In [41]:



df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1) 
    
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")


In [43]:
df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO

In [44]:
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

In [133]:
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18]
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)

<ipython-input-133-099c6972e96b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["EDAD"] = data.EDAD_ANIOS.apply(int)
<ipython-input-133-099c6972e96b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)


In [119]:
data_acumulada = data.groupby(["EDAD","SEXO","CAUSA_DEF"]).size().reset_index()

In [134]:
data_acumulada_nacional = data_acumulada.pivot(["EDAD","CAUSA_DEF"],"SEXO",0)

In [158]:
ef = data_acumulada_nacional.loc[17].sort_values(by=2,na_position='first').reset_index()

In [159]:
ef.CAUSA_DEF.map(dic_catminde)

0                          Enteritis debida a Salmonella
1      Otras gastroenteritis y colitis de origen infe...
2                               Fiebre de las trincheras
3                     Encefalitis viral, no especificada
4                          Encefalitis debida a varicela
                             ...                        
429                   Leucemia linfoblástica aguda [LLA]
430    Agresión con disparo de otras armas de fuego, ...
431    Persona lesionada en accidente de tránsito, de...
432    Lesión autoinfligida intencionalmente por ahor...
433                         COVID-19, virus identificado
Name: CAUSA_DEF, Length: 434, dtype: object

In [155]:
ef

SEXO,CAUSA_DEF,1,2,9
0,A090,NaN,1.0,NaN
1,A178,NaN,1.0,NaN
2,A410,NaN,1.0,NaN
3,A689,NaN,1.0,NaN
4,B159,NaN,1.0,NaN
...,...,...,...,...
360,X599,32.0,11.0,NaN
361,C910,38.0,26.0,NaN
362,X700,53.0,37.0,NaN
363,V892,78.0,15.0,NaN
